In [2]:
!pip install sqlalchemy

In [4]:
! pip install pymysql
! pip install pymongo
! pip install sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 20.4 MB/s eta 0:00:00


In [5]:
import pandas as pd
from sqlalchemy import create_engine, text

try:
  # 1) CSV 로드
  df = pd.read_csv("biometric_wearable_dataset.csv")
  df["timestamp"] = pd.to_datetime(df["timestamp"])


  # 2) MySQL 엔진 생성

  # 여기에 본인 Aiven 정보를 넣으세요
  host = "mysql-kimyoungjin-kimyoungjin8399.d.aivencloud.com"
  user = "avnadmin"
  password = "AVNS_TSuygyjaiMimnc7L5zZ"
  database = "defaultdb"
  port = 17507

  engine = create_engine(
      f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
  )

  # 3) CREATE TABLE
  create_table_sql = """
    CREATE TABLE IF NOT EXISTS biometric_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        timestamp DATETIME,
        heart_rate FLOAT,
        blood_oxygen FLOAT,
        body_temp FLOAT,
        resp_rate FLOAT,
        step_count INT,
        activity VARCHAR(50),
        accel_x FLOAT,
        accel_y FLOAT,
        accel_z FLOAT,
        gyro_x FLOAT,
        gyro_y FLOAT,
        gyro_z FLOAT,
        is_tampered INT
    );
    """

  with engine.connect() as conn:
      conn.execute(text(create_table_sql))
      conn.commit()

  print("✅ 테이블 검사 및 생성 완료")

  # 4) 데이터 INSERT
  df.to_sql(
      name="biometric_data",
      con=engine,
      if_exists="append",
      index=False
  )

  print("🎉 데이터 저장 완료!")

except Exception as e:
  print(e)

✅ 테이블 검사 및 생성 완료
🎉 데이터 저장 완료!


## [필수] 1) 고심박(high HR) 위험군 탐지
- 일정 기준(예: heart_rate > 150)을 넘는 순간을 찾아서
  - 발생 시각  
  - 해당 활동  
  - 발생 빈도를 분석한다.

In [38]:
from contextlib import closing

query = """
    SELECT
        DATE_FORMAT(timestamp, '%Y-%m-%d %H') AS formatted_date_min,
        activity,
        COUNT(*) AS high_hr_count
    FROM biometric_data
    WHERE heart_rate > 150
    GROUP BY formatted_date_min, activity
    ORDER BY formatted_date_min
"""

with closing(engine.raw_connection()) as raw_conn:
    with closing(raw_conn.cursor()) as cur:
        cur.execute(query)
        results = cur.fetchall()
        raw_conn.commit()

results

(('2024-04-01 00', 'Cycling', 27),
 ('2024-04-01 00', 'Resting', 25),
 ('2024-04-01 00', 'Running', 23),
 ('2024-04-01 00', 'Walking', 24),
 ('2024-04-01 01', 'Cycling', 14),
 ('2024-04-01 01', 'Resting', 13),
 ('2024-04-01 01', 'Running', 24),
 ('2024-04-01 01', 'Walking', 22),
 ('2024-04-01 02', 'Cycling', 15),
 ('2024-04-01 02', 'Resting', 21),
 ('2024-04-01 02', 'Running', 17),
 ('2024-04-01 02', 'Walking', 19),
 ('2024-04-01 03', 'Cycling', 24),
 ('2024-04-01 03', 'Resting', 26),
 ('2024-04-01 03', 'Running', 19),
 ('2024-04-01 03', 'Walking', 15),
 ('2024-04-01 04', 'Cycling', 19),
 ('2024-04-01 04', 'Resting', 23),
 ('2024-04-01 04', 'Running', 22),
 ('2024-04-01 04', 'Walking', 27),
 ('2024-04-01 05', 'Cycling', 19),
 ('2024-04-01 05', 'Resting', 23),
 ('2024-04-01 05', 'Running', 11),
 ('2024-04-01 05', 'Walking', 22),
 ('2024-04-01 06', 'Cycling', 12),
 ('2024-04-01 06', 'Resting', 21),
 ('2024-04-01 06', 'Running', 18),
 ('2024-04-01 06', 'Walking', 24),
 ('2024-04-01 07', '

## 2) 활동(activity)별 평균 생체지표 분석
- 활동별로 heart_rate, step_count, resp_rate, body_temp의 평균을 계산하고 비교한다.
- 어떤 활동이 가장 높은 생체 지표를 보이는지 분석한다.

In [39]:
from contextlib import closing

query = """
SELECT
	activity,
	ROUND(AVG(heart_rate),2) AS avg_heart_rate,
	ROUND(AVG(heart_rate),2) AS avg_heart_rate,
	ROUND(AVG(step_count),2) AS avg_step_count,
	ROUND(AVG(resp_rate),2) AS avg_resp_rate,
	ROUND(AVG(body_temp),2) AS avg_body_temp,
	ROUND(AVG(heart_rate + step_count + resp_rate +body_temp),2) AS avg_combined_score
FROM biometric_data
GROUP BY activity
ORDER BY avg_combined_score DESC
"""

# raw_connection() → pymysql connection
with closing(engine.raw_connection()) as raw_conn:
    with closing(raw_conn.cursor()) as cur:
        cur.execute(query)
        results = cur.fetchall()
        raw_conn.commit()

# 보기 좋게 출력
for row in results:
    print(row)

('Walking', 119.29, 119.29, Decimal('101.55'), 18.07, 37.53, 276.45)
('Resting', 121.39, 121.39, Decimal('98.80'), 18.18, 37.59, 275.96)
('Running', 118.23, 118.23, Decimal('98.95'), 18.09, 37.58, 272.85)
('Cycling', 119.14, 119.14, Decimal('97.95'), 17.93, 37.43, 272.45)


## 3) 체온 이상치(high fever) 탐지
- body_temp ≥ 38.0°C인 구간을 찾아  
  - 어떤 활동에서 많이 나타나는지  
  - 위험 패턴이 존재하는지 분석한다.

In [42]:
from contextlib import closing

query = """
    SELECT
        activity,
        COUNT(*) AS high_fever_count
    FROM biometric_data
    WHERE body_temp >= %s
    GROUP BY activity
    ORDER BY high_fever_count DESC;
"""

with closing(engine.raw_connection()) as raw_conn:
    with closing(raw_conn.cursor()) as cur:
        cur.execute(query, (38.0,))
        results = cur.fetchall()
        raw_conn.commit()

for row in results:
    print(row)

('Walking', 454)
('Running', 452)
('Resting', 430)
('Cycling', 401)


## 4) 산소포화도(SpO₂) 저하 위험 탐지
- blood_oxygen < 94% 데이터를 필터링하여  
  - 발생 시각  
  - activity를 분석한다.

In [41]:
from contextlib import closing

query = """
SELECT
 	DATE_FORMAT(timestamp, '%Y-%m-%d %H:%i') AS formatted_date_min,
 	activity,
 	blood_oxygen
 FROM biometric_data
 WHERE blood_oxygen>=94
 ORDER BY timestamp
"""

with closing(engine.raw_connection()) as raw_conn:
    with closing(raw_conn.cursor()) as cur:
        cur.execute(query)
        results = cur.fetchall()
        raw_conn.commit()

results



(('2024-04-01 00:00', 'Resting', 97.7048),
 ('2024-04-01 00:00', 'Walking', 96.1682),
 ('2024-04-01 00:00', 'Cycling', 95.9864),
 ('2024-04-01 00:00', 'Walking', 96.8389),
 ('2024-04-01 00:01', 'Running', 97.948),
 ('2024-04-01 00:01', 'Walking', 95.3352),
 ('2024-04-01 00:01', 'Walking', 98.7257),
 ('2024-04-01 00:01', 'Running', 95.0764),
 ('2024-04-01 00:02', 'Running', 98.0797),
 ('2024-04-01 00:02', 'Walking', 97.2341),
 ('2024-04-01 00:03', 'Running', 95.6035),
 ('2024-04-01 00:03', 'Walking', 95.2057),
 ('2024-04-01 00:03', 'Cycling', 95.7712),
 ('2024-04-01 00:04', 'Walking', 96.3409),
 ('2024-04-01 00:04', 'Running', 99.8464),
 ('2024-04-01 00:04', 'Resting', 94.2974),
 ('2024-04-01 00:04', 'Resting', 94.2483),
 ('2024-04-01 00:04', 'Running', 94.1705),
 ('2024-04-01 00:05', 'Walking', 96.638),
 ('2024-04-01 00:05', 'Running', 94.8983),
 ('2024-04-01 00:05', 'Walking', 95.1743),
 ('2024-04-01 00:05', 'Cycling', 94.3236),
 ('2024-04-01 00:05', 'Resting', 109.855),
 ('2024-04-01

## 6) 센서 오류(is_tampered) 데이터 탐지
- is_tampered = 1 이 발생한 횟수를 파악하고  
- 오류가 어떤 활동에서 많이 발생하는지 분석한다.


In [28]:
from contextlib import closing

query = """
    SELECT
        activity,
        COUNT(*) AS tampered_count
    FROM biometric_data
    WHERE is_tampered = 1
    GROUP BY activity
    ORDER BY tampered_count DESC;
"""

with closing(engine.raw_connection()) as raw_conn:
    with closing(raw_conn.cursor()) as cur:
        cur.execute(query)
        results = cur.fetchall()
        raw_conn.commit()

results

(('Walking', 198), ('Running', 190), ('Resting', 187), ('Cycling', 185))

## 7) 시간대별 생체지표 분석
- 시간(HOUR) 기준으로 데이터를 그룹화하여  
  - 시간대별 평균 heart_rate  
  - 시간대별 평균 body_temp 패턴을 분석한다.

In [43]:
from contextlib import closing

query = """
SELECT
	HOUR(timestamp) AS hour_data,
	ROUND(AVG(heart_rate),2),
	ROUND(AVG(body_temp),2)
FROM biometric_data
GROUP BY hour_data
"""

with closing(engine.raw_connection()) as raw_conn:
    with closing(raw_conn.cursor()) as cur:
        cur.execute(query)
        results = cur.fetchall()
        raw_conn.commit()

results

((0, 121.74, 37.63),
 (1, 117.57, 37.8),
 (2, 116.86, 37.48),
 (3, 119.79, 37.5),
 (4, 121.59, 37.55),
 (5, 119.73, 37.56),
 (6, 118.77, 37.61),
 (7, 120.7, 37.55),
 (8, 120.21, 37.38),
 (9, 117.54, 37.5),
 (10, 120.17, 37.52),
 (11, 118.8, 37.33),
 (12, 120.08, 37.71),
 (13, 119.35, 37.36))